# Sell high, buy low using battery storage
Exploring the prospect of using battery storage to store some of the energy produced when prices are low to then sell it for a higher price later.

To keep things simple, I'm making a few simplifying assumptions:
- Any energy stored today needs to be sold by the end of tomorrow (although more energy can be stored at tomorrows price so effectively this rule does nothing)
- Ignore costs due to battery cycling. Batteries can undergo a finite number of charge-discharge cycles, after which they need to be replaced, but I'm ignoring any costs due to this (and these costs are likely to be high)
- If predicted price for tomorrow is higher than for today, I'm fully filling up the batteries (if the solar arrays aren't producing, just get electricity from the grid).

## Excecutive summary
- The cost for a 70 MWh battery in 2022 is 12 million AUD at a minimum
- With perfect information on tomorrows prices, could make 300,000 AUD per year based on data for 2015-2020
- The maximum return on investment is then 2.6% yearly, ignoring depreciation of batteries over time
- Using an out of the box gradient boosted tree to predict when to fill up the batteries, can reach 40% of the maximum profits - this can probably be significantly improved, but that won't change the conclusions about profitability.
- Possibly could make higher profits by selling during time of day when demand is peaking?

## Import packages

In [1]:
from typing import Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import sem
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from lightgbm import LGBMRegressor
plt.style.use('seaborn-deep')
plt.rcParams['figure.figsize'] = (16,9)
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

## Import and preprocess data

In [2]:
def load_data():
    df = pd.read_csv('../data/raw/energy_demand.csv')
    column_mapper = {"RRP":"price", "demand_pos_RRP":"demand_pos_price", "RRP_positive":"price_positive", "demand_neg_RRP":"demand_neg_price", "RRP_negative":"price_negative", "frac_at_neg_RRP":"frac_neg_price"}
    df.rename(columns = column_mapper, inplace = True)

    # Convert datatypes
    df.date = pd.to_datetime(df.date)
    df.school_day = df.school_day.map({"N": False, "Y":True}).astype('bool')
    df.holiday = df.holiday.map({"N": False, "Y":True}).astype('bool')

    # Extract year, month and day of week from data
    df['year'] = df.date.dt.year
    df['month'] = df.date.dt.month
    df['dow'] = df.date.dt.day_of_week
    df['week'] = df.date.dt.isocalendar().week.astype('int')

    # Convert solar exposure from MJ/m^2 to MWh/m^2 (1 MJ = 1/(60*60) MWh)
    df.solar_exposure = df.solar_exposure/3600

    # Set date as index so can do resampling using pandas
    df.set_index('date', inplace=True)
    
    return df

In [3]:
df = load_data()

## Cost for a 70 MWh battery
The cost per unit capacity for lithium ion batteries is something like 120-350 USD/kWh = 170-490 AUD/kWh ([National Renewable Energy Laboratory](https://www.nrel.gov/docs/fy21osti/79236.pdf), estimate for [Tesla](https://www.forbes.com/sites/greatspeculations/2021/12/01/are-battery-cost-improvements-still-a-big-driver-of-teslas-margins/?sh=447fbdcb4ae7), [Wikipedia](https://en.wikipedia.org/wiki/Sodium-ion_battery#Advantages_and_disadvantages_over_other_battery_technologies)), so a 70 MWh battery would cost around 12 - 34 million AUD.

In [4]:
battery_price_per_kWh = 170
battery_price = 70000*battery_price_per_kWh/1e6
print(f"Cost for 70 MWh battery at {battery_price_per_kWh:.1f} AUD/kWh is {battery_price:.2f} million AUD")

Cost for 70 MWh battery at 170.0 AUD/kWh is 11.90 million AUD


## How well is it possible to do in theory?
If we were able to predict the future perfectly, how much money could we make using a 70 MWh battery system? The strategy would be to fill up the batteries whenever today's price is lower than tomorrow's.

Calculate the profit that would be made everyday:

In [5]:
df['price_tomorrow'] = df.price.shift(-1,'D')
df['fill_battery'] = df.price_tomorrow > df.price
df['profit'] = (df.price_tomorrow - df.price)*df.fill_battery*70

Total profit over the entire time period is then roughly \\$1.8M, which corresponds to a daily profit of roughly 860 AUD (314,000 AUD yearly)

In [6]:
total_profit = df.profit.sum()/1e6
print(f"Total maximum profit over time period in data: {total_profit:.2f} million AUD")

daily_profit = df.profit.mean()
print(f"Mean daily profit over time period in data: {daily_profit:.0f} AUD ({daily_profit*365/1e3:.1f}k AUD yearly)")

annual_ROI = daily_profit*365/(battery_price*1e6)
print(f"Maximal yearly ROI for battery: {annual_ROI*100:.1f}%")

Total maximum profit over time period in data: 1.81 million AUD
Mean daily profit over time period in data: 861 AUD (314.3k AUD yearly)
Maximal yearly ROI for battery: 2.6%


The **maximum yearly return on investment is thus approximately 2.6%**, which is very low compared to typical returns for stocks and comparable to government bonds, with the caveat that the calculation here isn't taking into account account the depreciation of the batteries (which would further reduce the ROI). It would take the batteries roughly **40 years to pay back for themselves**, and it seems unlikely they would remain functional for that long (at least based on my personal experience with cell phone batteries...)

## To charge or not to charge - how well can we do in practice?
In practice we don't have perfect information about tomorrow's electricity prices, so we are probably not able to reach the theoretical maximum profit for the battery system. Using ***data science*** it may be possible to predict whether or not tomorrow's price is higher than today's and make the decision to fill or not to fill the batteries based on a predictive model. In this section, I'm going to train a simple linear regression model to predict the price of electricity on a given day, provided the following information:
- Price of electricity for the past 7 days. We would probably have access to this data by the time we have to decide if we want to charge the batteries for tomorrow (current days prices might not be available in practice, but I'll assume they are for now).
- Weather info: min and max temperature, rainfall and solar exposure for the day. Weather forecasts are typically quite accurate for the next day, so we would have access to good approximations of this information.
- Historic electricity demand for the given day
- What day of the week is the day we are trying to predict, and is it a holiday

### Generate data
Generating the data needed for training a model

In [7]:
def generate_model_data(df: pd.DataFrame, N_prev = 7, N_folds = 5) -> pd.DataFrame:
    """
    Generates data that is used to train a model to predict electricity prices on a given day.
    """
    columns = ['demand',
               'price',
               'min_temperature',
               'max_temperature',
               'solar_exposure',
               'rainfall',
               'school_day',
               'holiday',
               'week']
    df = df.copy()[columns]
    
    # Get prices for N_prev days and store them as new columns
    for n in range(1,N_prev+1):
        df[f'price_{n}'] = df.price.shift(n,'D')
    
    df.attrs['N_prev'] = N_prev
        
    # Remove dates where previous prices are not available
    df.dropna(inplace = True)
    
    # Split data into folds for use in cross validation
    date_max = df.index.max()
    day = pd.Timedelta(1,'D')
    df['fold'] = 0
    for i, n in enumerate(range(N_folds,0,-1)):
        df.loc[date_max-(i+1)*100*day:date_max-i*100*day, 'fold'] = n
        
    df.reset_index(inplace = True)
        
    return df

def generate_train_test(df: pd.DataFrame, fold: int) -> Tuple[pd.DataFrame]:
    """
    Generates train and test datasets for a given test fold
    """
    df = df.copy()
    
    train = df[df.fold < fold]
    y_train = train.pop('price')
    X_train = train.drop(columns = ['demand','fold'])
    
    test = df[df.fold == fold]
    y_test = test.pop('price')
    X_test = test.drop(columns = ['demand', 'fold'])

    return X_train, y_train, X_test, y_test

def get_X_y(df: pd.DataFrame) -> Tuple[pd.DataFrame]:
    """
    Separates dataframe to features and target.
    """
    
    y = df.pop('price')
    X = df.drop(columns = ['demand', 'fold'])
    
    return X, y

### Transform data
Define a pipeline for transforming data

In [10]:
def transform_data(df: pd.DataFrame, fold: int) -> pd.DataFrame:
    """
    Transforms the input data to be fit by the predictive model
    """
    df = calc_weekly_median(df)
    df = calc_historical_price(df, fold)    
    df = calc_historical_demand(df, fold)
    
    df.set_index('date', inplace = True)

    return df
    
def calc_weekly_median(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the median price of electricity for the past 7 days for each date in data.
    """
    cols = []
    for n in range(1,df.attrs['N_prev']+1):
        cols.append(f'price_{n}')
        
    df['median_price'] = df[cols].median(axis = 1)
    
    return df
    
def calc_historical_price(df: pd.DataFrame, fold: int) -> pd.DataFrame:
    """
    Calculates median historical price by week for each date using only data that is before the validation period (but not using data from future).
    """
    df = df.merge(df[df.fold < fold].groupby('week').price.median().to_frame('historical_median').reset_index(), on = 'week')
    return df

def calc_historical_demand(df: pd.DataFrame, fold: int) -> pd.DataFrame:
    """
    Calculates median historical demand by week for each date in data (but not using data from future).
    """
    df = df.merge(df[df.fold < fold].groupby('week').demand.median().to_frame('historical_demand').reset_index(), on = 'week')
    return df

### Set up model

In [11]:
def fit_predict_lin_reg(X_train: pd.DataFrame, y_train: pd.DataFrame, X_test: pd.DataFrame) -> pd.DataFrame:
    """
    Sets up a linear regression model, fits it to data and makes predictions.
    """
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    preds = X_test.copy()
    preds['predicted_price'] = y_pred
    preds['predicted_buy'] = (preds.predicted_price - preds.price_1) > 0

    return preds

def fit_predict_LGBM(X_train: pd.DataFrame, y_train: pd.DataFrame, X_test: pd.DataFrame, params: dict = {}) -> pd.DataFrame:
    """
    Sets up a linear regression model, fits it to data and makes predictions.
    """
    model = LGBMRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    preds = X_test.copy()
    preds['predicted_price'] = y_pred
    preds['predicted_buy'] = (preds.predicted_price - preds.price_1) > 0

    return preds

### Metrics

In [15]:
def correct_buy_percentage(y_true, y_pred) -> float:
    """
    Calculates the percentage of days where the model was able to corectly predict
    if the battery should be filled or not.
    """
    return np.mean(y_true==y_pred)

def percentage_of_max_profit(df) -> float:
    """
    Calculates the percentage of the max profit that was obtained.
    """
    return df.profit.sum()/df.max_profit.sum()

### Cross validation

In [16]:
def cross_validate_LR(N_folds: int = 5):
    """
    Repeats the data generation, training, and testing of the model for
    different folds.
    """
    df = load_data()
    data_all_folds = generate_model_data(df)
    
    result_dict = {}
    result_dict['mape'] = []
    result_dict['cbp'] = []
    result_dict['pomp'] = []
    
    # Loop over the folds
    for fold in range(1, N_folds+1):
        data = transform_data(data_all_folds, fold)
        X_train, y_train, X_test, y_test = generate_train_test(data, fold)
        preds = fit_predict_lin_reg(X_train, y_train, X_test)
        
        # Calculate data needed for calculating metrics
        preds['true_price'] = y_test
        preds['true_buy'] = (preds.true_price - preds.price_1) > 0
        preds['profit'] = (preds.true_price - preds.price_1)*preds.predicted_buy*70
        preds['max_profit'] = (preds.true_price - preds.price_1)*preds.true_buy*70
        
        # Calculate metrics
        mape = mean_absolute_percentage_error(preds.true_price, preds.predicted_price)
        cbp = correct_buy_percentage(preds.true_buy, preds.predicted_buy)
        pomp = percentage_of_max_profit(preds)
        
        result_dict['mape'].append(mape)
        result_dict['cbp'].append(cbp)
        result_dict['pomp'].append(pomp)
    
    return result_dict

def cross_validate_LGBM(N_folds: int = 5):
    """
    Repeats the data generation, training, and testing of the model for
    different folds.
    """
    df = load_data()
    data_all_folds = generate_model_data(df)
    
    result_dict = {}
    result_dict['mape'] = []
    result_dict['cbp'] = []
    result_dict['pomp'] = []
    
    # Loop over the folds
    for fold in range(1, N_folds+1):
        data = transform_data(data_all_folds, fold)
        X_train, y_train, X_test, y_test = generate_train_test(data, fold)
        preds = fit_predict_LGBM(X_train, y_train, X_test)
        
        # Calculate data needed for calculating metrics
        preds['true_price'] = y_test
        preds['true_buy'] = (preds.true_price - preds.price_1) > 0
        preds['profit'] = (preds.true_price - preds.price_1)*preds.predicted_buy*70
        preds['max_profit'] = (preds.true_price - preds.price_1)*preds.true_buy*70
        
        # Calculate metrics
        mape = mean_absolute_percentage_error(preds.true_price, preds.predicted_price)
        cbp = correct_buy_percentage(preds.true_buy, preds.predicted_buy)
        pomp = percentage_of_max_profit(preds)
        
        result_dict['mape'].append(mape)
        result_dict['cbp'].append(cbp)
        result_dict['pomp'].append(pomp)
    
    return result_dict

def show_summary_stats(results_dict: dict):
    """
    Print the the mean and error in mean accross different folds
    """
    for metric, values in results_dict.items():
        mean = np.mean(values)
        std_err = sem(values)
        print(f"Mean for {metric}: {mean:.3f}+/-{std_err:.3f}")

In [17]:
show_summary_stats(cross_validate_LGBM())

Mean for mape: 0.394+/-0.078
Mean for cbp: 0.621+/-0.013
Mean for pomp: 0.412+/-0.026
